<a target="_blank" href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/2-Advance/FinRL_Ensemble_StockTrading_ICAIF_2020.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
# # ## install finrl library
# !pip install wrds
# !pip install swig
# # !pip install -q condacolab
# # import condacolab
# # condacolab.install()
# !apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git


In [2]:
# !pip install yfinance pandas numpy matplotlib stockstats gym stable-baselines3 tensorflow pyfolio

In [3]:
# !pip install pandas_market_calendars

<a id='1.3'></a>
## 2.3. Import Packages

In [3]:
import warnings
warnings.filterwarnings("ignore")

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import statistics 
from stable_baselines3.common.callbacks import BaseCallback

%matplotlib inline
from FinRL.finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from FinRL.finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from FinRL.finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from FinRL.finrl.agents.stablebaselines3.models import DRLAgent,DRLEnsembleAgent
from FinRL.finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from stable_baselines3.common.vec_env import DummyVecEnv
from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

In [5]:

df_account_value = pd.DataFrame()
for i in range(126, 1134+1, 63+63):
    temp = pd.read_csv(f'acc/account_value_trade_ensemble_{i}.csv')
    df_account_value = pd.concat([df_account_value, temp], ignore_index=True)

sharpe = (252 ** 0.5) * df_account_value.account_value.pct_change(1).mean() / df_account_value.account_value.pct_change(1).std()
print('Sharpe Ratio: ', sharpe)



Sharpe Ratio:  -0.3822855217131029


In [6]:
# df_account_value = pd.read_csv('saved_data/df_account_value.csv', index_col=0)
df_account_value_a2c = pd.read_csv('acc/df_account_value_a2c.csv', index_col=0)
df_account_value_ppo = pd.read_csv('acc/df_account_value_ppo.csv', index_col=0)

In [7]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)

==============Get Backtest Results===========
Annual return         -0.123643
Cumulative returns    -0.256925
Annual volatility      0.255997
Sharpe ratio          -0.382286
Calmar ratio          -0.274382
Stability              0.658178
Max drawdown          -0.450623
Omega ratio            0.921164
Sortino ratio         -0.465027
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.914373
Daily value at risk   -0.032641
dtype: float64


In [8]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ppo)
perf_stats_all = pd.DataFrame(perf_stats_all)
# ==============Get Backtest Results===========
# Annual return               0.379725
# Cumulative returns          0.083797
# Annual volatility      338567.947564
# Sharpe ratio                2.017211
# Calmar ratio                0.379725
# Stability                   0.324152
# Max drawdown               -1.000000
# Omega ratio             22751.286315
# Sortino ratio          155782.345252
# Skew                             NaN
# Kurtosis                         NaN
# Tail ratio                  9.187028
# Daily value at risk    -39945.381418

==============Get Backtest Results===========
Annual return          0.006777
Cumulative returns     0.031660
Annual volatility      0.220004
Sharpe ratio           0.141148
Calmar ratio           0.012462
Stability              0.125431
Max drawdown          -0.543809
Omega ratio            1.027308
Sortino ratio          0.191905
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.869766
Daily value at risk   -0.027595
dtype: float64


In [9]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_a2c)
perf_stats_all = pd.DataFrame(perf_stats_all)
# ==============Get Backtest Results===========
# Annual return               0.379725
# Cumulative returns          0.083797
# Annual volatility      338567.947564
# Sharpe ratio                2.017211
# Calmar ratio                0.379725
# Stability                   0.324152
# Max drawdown               -1.000000
# Omega ratio             22751.286315
# Sortino ratio          155782.345252
# Skew                             NaN
# Kurtosis                         NaN
# Tail ratio                  9.187028
# Daily value at risk    -39945.381418

==============Get Backtest Results===========
Annual return          0.042300
Cumulative returns     0.210704
Annual volatility      0.206109
Sharpe ratio           0.304811
Calmar ratio           0.099497
Stability              0.402325
Max drawdown          -0.425138
Omega ratio            1.056050
Sortino ratio          0.409146
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.963487
Daily value at risk   -0.025718
dtype: float64


In [10]:
#baseline stats
print("==============Get Baseline Stats===========")
df_dji_ = get_baseline(
        ticker="^DJI",
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(df_dji_, value_col_name = 'close')

==============Get Baseline Stats===========
YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)


[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (1080, 8)
Annual return          0.064593
Cumulative returns     0.307681
Annual volatility      0.150772
Sharpe ratio           0.490850
Calmar ratio           0.294399
Stability              0.568673
Max drawdown          -0.219408
Omega ratio            1.090247
Sortino ratio          0.702753
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.963434
Daily value at risk   -0.018702
dtype: float64


<a id='6.2'></a>
## 7.2 BackTestPlot

In [25]:
import numpy as np
import pandas as pd
import cvxpy as cp
processed = pd.read_csv('acc/processed.csv', index_col=0)
stock_dimension = len(processed.tic.unique())
df_result_ensemble = pd.DataFrame({'date': df_account_value['date'], 
                                 'ensemble': df_account_value['account_value']}).set_index('date')

df_result_a2c = pd.DataFrame({'date': df_account_value_a2c['date'], 
                             'a2c': df_account_value_a2c['account_value']}).set_index('date')

df_result_ppo = pd.DataFrame({'date': df_account_value_ppo['date'], 
                             'ppo': df_account_value_ppo['account_value']}).set_index('date')

# Merge all DataFrames using their date index
result = df_result_ensemble.join([df_result_ppo, df_result_a2c])


TRAIN_START_DATE = '2014-01-01'
TRAIN_END_DATE = '2020-09-30'
TEST_START_DATE = '2020-09-30'
TEST_END_DATE = '2025-06-01'
# Helper function to process dataframe for portfolio optimization
def process_df_for_mvo(df):
    # Sort and filter the DataFrame
    df = df.sort_values(['date', 'tic'], ignore_index=True)[['date', 'tic', 'close']]
    
    # Get the complete list of unique tickers
    all_tickers = df['tic'].unique().tolist()
    
    # Initialize mvo with all tickers
    mvo = pd.DataFrame(columns=all_tickers)

    for i in range(df.shape[0] // stock_dimension):
        n = df.iloc[i * stock_dimension : (i + 1) * stock_dimension, :]
        date = n['date'].iloc[0]  # Use .iloc to ensure single date
        
        # Create a dictionary to handle missing tickers
        row_data = {ticker: 0 for ticker in all_tickers}  # Default value for missing tickers
        row_data.update(dict(zip(n['tic'], n['close'])))  # Update with available tickers
        
        # Add the row to mvo
        mvo.loc[date] = row_data

    return mvo

# Helper function to compute stock returns (in decimal form)
def StockReturnsComputing(StockPrice, Rows, Columns):
    StockReturn = np.zeros([Rows - 1, Columns])
    for j in range(Columns):
        for i in range(Rows - 1):
            StockReturn[i, j] = (StockPrice[i + 1, j] - StockPrice[i, j]) / StockPrice[i, j]
    return StockReturn

# Assuming data_split, processed_full, TRAIN_START_DATE, TRAIN_END_DATE, TEST_START_DATE, TEST_END_DATE, and stock_dimension are defined
train_mvo = data_split(processed, TRAIN_START_DATE, TRAIN_END_DATE).reset_index()
trade_mvo = data_split(processed, TEST_START_DATE, TEST_END_DATE).reset_index()


In [26]:
train_mvo

,index,date,close,high,low,open,volume,tic
0,0,2014-01-02,2.29,2.29,2.28,2.29,146982,ACB
1,0,2014-01-02,29.05,29.36,28.98,29.05,87000,BVH
2,0,2014-01-02,9.01,9.07,8.96,9.07,156420,CTG
3,0,2014-01-02,6.45,6.50,6.42,6.48,207470,FPT
4,0,2014-01-02,31.69,31.69,31.21,31.69,229950,GAS
...,...,...,...,...,...,...,...,...
37276,1680,2020-09-29,81.33,83.56,81.33,83.56,394490,VIC
37277,1680,2020-09-29,104.00,105.40,104.00,105.40,374370,VJC
37278,1680,2020-09-29,86.17,88.85,85.22,86.80,5191680,VNM
37279,1680,2020-09-29,7.75,8.00,7.73,7.83,11580400,VPB


In [27]:
trade_mvo

,index,date,close,high,low,open,volume,tic
0,0,2020-09-30,8.44,8.44,8.33,8.40,3624221,ACB
1,0,2020-09-30,37.95,38.04,37.25,37.29,65010,BCM
2,0,2020-09-30,23.29,23.34,23.11,23.20,933550,BID
3,0,2020-09-30,42.34,42.43,41.56,41.73,471690,BVH
4,0,2020-09-30,17.72,17.85,17.52,17.75,4764680,CTG
...,...,...,...,...,...,...,...,...
34741,1162,2025-05-30,98.40,99.00,96.30,97.10,3270500,VIC
34742,1162,2025-05-30,87.00,87.70,86.60,87.10,531400,VJC
34743,1162,2025-05-30,54.90,55.40,54.80,55.40,3273900,VNM
34744,1162,2025-05-30,17.95,18.30,17.90,18.25,11859100,VPB


In [ ]:

initial_amount = 100000000

In [ ]:
StockData = process_df_for_mvo(train_mvo)
TradeData = process_df_for_mvo(trade_mvo)
  

ACB    0
BVH    0
CTG    0
FPT    0
GAS    0
HPG    0
MBB    0
MSN    0
SHB    0
SSI    0
STB    0
VCB    0
VHM    0
VIC    0
VNM    0
BID    0
MWG    0
SAB    0
VIB    0
VJC    0
PLX    0
VPB    0
LPB    0
VRE    0
HDB    0
BCM    0
GVR    0
TPB    0
TCB    0
dtype: int64
               ACB          BVH          CTG          FPT          GAS  \
count  1242.000000  1242.000000  1242.000000  1242.000000  1242.000000   
mean      5.253132    53.719026    13.432818    15.551932    49.104130   
std       2.012180    16.924565     3.212001     5.363385    15.381549   
min       2.280000    24.470000     8.060000     6.430000    16.090000   
25%       3.050000    41.232500    10.990000    10.225000    35.502500   
50%       6.120000    49.270000    13.410000    15.635000    49.660000   
75%       6.810000    66.900000    15.272500    19.022500    63.207500   
max       9.860000    91.360000    25.160000    25.990000    81.040000   

               HPG          MBB          MSN          SHB  

In [33]:
StockData = StockData[StockData > 0]  
StockData = StockData.fillna(method='ffill').fillna(method='bfill')  # Forward and backward fill # Remove rows with zero or negative prices

In [34]:
arStockPrices = np.asarray(StockData)
Rows, Cols = arStockPrices.shape
arReturns = StockReturnsComputing(arStockPrices, Rows, Cols)
covReturns = np.cov(arReturns, rowvar=False)
if np.any(np.isnan(covReturns)):
    print("Covariance matrix still contains NaN values. Further investigation needed.")
else:
    print("Covariance matrix is clean.")

Covariance matrix is clean.


In [39]:
# Process data for training and trading periods
  # Check for any anomalies in the data
# Get tickers
tickers = StockData.columns.tolist()
print("StockData tickers:", tickers)
print("Number of tickers:", len(tickers))

# Compute asset returns
arStockPrices = np.asarray(StockData)
Rows, Cols = arStockPrices.shape
arReturns = StockReturnsComputing(arStockPrices, Rows, Cols)
print("arReturns shape:", arReturns.shape)

# Compute mean returns and covariance matrix
meanReturns = np.mean(arReturns, axis=0)
covReturns = np.cov(arReturns, rowvar=False)
print("covReturns shape:", covReturns.shape)

# Check for NaN in covariance matrix
if np.any(np.isnan(covReturns)):
    print("Warning: covReturns contains NaN values. Check input data for missing or invalid prices.")
    raise ValueError("NaN values in covariance matrix")

# Convert meanReturns and covReturns to pandas with tickers
meanReturns = pd.Series(meanReturns, index=tickers)
covReturns = pd.DataFrame(covReturns, index=tickers, columns=tickers)

# ── Replace pypfopt EfficientFrontier with CVXPY-based Min Variance ─────────────
# 1) Extract Σ (covariance matrix) in the correct ticker order
Σ = covReturns.loc[tickers, tickers].to_numpy()  # shape (n, n)
n = len(tickers)

# 2) Define CVXPY variable
w = cp.Variable(n)

# 3) Define objective: minimize w^T Σ w
objective = cp.Minimize(cp.quad_form(w, Σ))

# 4) Define constraints: sum(w) == 1, 0 <= w_i <= 0.5
constraints = [
    cp.sum(w) == 1,
    w >= 0,
    w <= 0.5
]

# 5) Formulate and solve the problem
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.SCS)

# 6) Check solver status
if prob.status not in ["optimal", "optimal_inaccurate"]:
    raise ValueError(f"CVXPY did not find an optimal solution (status={prob.status})")

# 7) Extract raw weights and clean them (clamp negatives to zero, re-normalize)
raw_weights = w.value
cleaned = np.maximum(raw_weights, 0)
cleaned = cleaned / cleaned.sum()

# 8) Map cleaned weights back into a dict keyed by ticker
cleaned_weights_min_var = {tickers[i]: float(cleaned[i]) for i in range(n)}

# Debug: Check weights
print("Cleaned weights keys:", list(cleaned_weights_min_var.keys()))
print("Number of weights:", len(cleaned_weights_min_var))

# Verify ticker alignment
missing_tickers = [tic for tic in tickers if tic not in cleaned_weights_min_var]
if missing_tickers:
    print("Error: Missing tickers in weights:", missing_tickers)
    raise KeyError(f"Tickers {missing_tickers} not found in cleaned_weights_min_var")

# Compute dollar allocations ($1,000,000 total portfolio value)
min_var_weights = np.array([100_000_000 * cleaned_weights_min_var[tic] for tic in tickers])

# Compute number of shares based on last training period prices
# (assuming fractional shares are allowed for backtesting)
LastPrice = np.array([1 / p for p in StockData.tail(1).to_numpy()[0]])
Initial_Portfolio_min_var = np.multiply(min_var_weights, LastPrice)

# Calculate portfolio value over the trade period
# Ensure TradeData columns align with `tickers` order
TradeData = TradeData[tickers]
Portfolio_Assets_min_var = TradeData @ Initial_Portfolio_min_var

# Create DataFrame for Min Variance results
df_account_value_min_var = pd.DataFrame({
    'date': TradeData.index,
    'account_value': Portfolio_Assets_min_var
}).reset_index(drop=True)

# Prepare result DataFrame for comparison
df_result_min_var = pd.DataFrame({
    'date': df_account_value_min_var['date'],
    'min_var': df_account_value_min_var['account_value']
}).set_index('date')

# Assuming df_result_ensemble, df_result_ppo, df_result_a2c, and df_dji are defined
# Merge all strategy results
result = df_result_ensemble.join([df_result_ppo, df_result_a2c, df_result_min_var], how='inner')

# Calculate cumulative returns
initial_ensemble = result['ensemble'].iloc[0]
initial_ppo = result['ppo'].iloc[0]
initial_a2c = result['a2c'].iloc[0]
initial_min_var = result['min_var'].iloc[0]


df_cumulative = pd.DataFrame({
    'Ensemble': (result['ensemble'] / initial_ensemble - 1) * 100,
    'PPO':      (result['ppo']      / initial_ppo      - 1) * 100,
    'A2C':      (result['a2c']      / initial_a2c      - 1) * 100,
    'Min Var':  (result['min_var']  / initial_min_var  - 1) * 100,
}, index=result.index)


StockData tickers: ['ACB', 'BVH', 'CTG', 'FPT', 'GAS', 'HPG', 'MBB', 'MSN', 'SHB', 'SSI', 'STB', 'VCB', 'VHM', 'VIC', 'VNM', 'BID', 'MWG', 'SAB', 'VIB', 'VJC', 'PLX', 'VPB', 'LPB', 'VRE', 'HDB', 'BCM', 'GVR', 'TPB', 'TCB']
Number of tickers: 29
arReturns shape: (1241, 29)
covReturns shape: (29, 29)
Cleaned weights keys: ['ACB', 'BVH', 'CTG', 'FPT', 'GAS', 'HPG', 'MBB', 'MSN', 'SHB', 'SSI', 'STB', 'VCB', 'VHM', 'VIC', 'VNM', 'BID', 'MWG', 'SAB', 'VIB', 'VJC', 'PLX', 'VPB', 'LPB', 'VRE', 'HDB', 'BCM', 'GVR', 'TPB', 'TCB']
Number of weights: 29


In [40]:
df_cumulative

,Ensemble,PPO,A2C,Min Var
date,,,,
2020-12-29,0.000000,0.000000,0.000000,0.000000
2020-12-30,0.076040,-0.710751,0.341504,1.275185
2020-12-31,0.308843,0.293325,1.858788,3.045408
2021-01-05,1.036818,2.847952,3.651898,4.136219
2021-01-06,0.967603,4.317502,5.032917,4.082920
...,...,...,...,...
2025-04-11,-25.775101,-11.971634,-15.997561,26.873269
2025-04-14,-25.277952,-11.925259,-15.270972,24.095700
2025-04-15,-25.297419,-11.931465,-15.310037,22.084847


In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# … all your data-prep up through df_cumulative …
# ensure the index is parsed as real timestamps
df_cumulative.index = pd.to_datetime(df_cumulative.index)

# Create a figure and axes explicitly
fig, ax = plt.subplots(figsize=(15, 5))

# Plot the cumulative returns (this draws your four lines exactly as before)
df_cumulative.plot(ax=ax)

# —— now override the default x-axis ticks & grid —— 

# 1. Major ticks every quarter (Jan, Apr, Jul, Oct)
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1,4,7,10]))

# 2. Format each tick as “Jan 4” on line 1, “2016” on line 2
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d\n%Y'))

# 3. Draw vertical gridlines at those quarter-points, plus horizontal at y-majors
ax.grid(axis='x', which='major', linestyle='--', linewidth=0.5)
ax.grid(axis='y', which='major', linestyle='--', linewidth=0.5)

# 4. Tweak tick label alignment
plt.setp(ax.get_xticklabels(), rotation=0, ha='center')

# —— finish off your labels/legend/title exactly as before —— 

ax.set_title('Cumulative Return Comparison of Trading Strategies vith Transaction Cost')
ax.set_xlabel('Date')
ax.set_ylabel('Cumulative Return (%)')
ax.legend(['Ensemble', 'PPO', 'A2C', 'Min var'])
plt.tight_layout()
plt.show()
plt.savefig('cumulative_return_newa2cppo_vn.pdf')


In [44]:
processed

,date,close,high,low,open,volume,tic
0,2014-01-02,2.29,2.29,2.28,2.29,146982,ACB
1,2014-01-03,2.31,2.31,2.28,2.29,98656,ACB
2,2014-01-06,2.29,2.31,2.28,2.28,190009,ACB
3,2014-01-07,2.32,2.35,2.29,2.31,152905,ACB
4,2014-01-08,2.31,2.32,2.29,2.32,58852,ACB
...,...,...,...,...,...,...,...
72022,2025-05-26,25.50,25.65,24.50,25.60,8734200,VRE
72023,2025-05-27,25.30,25.75,25.25,25.50,11529600,VRE
72024,2025-05-28,26.70,26.80,25.35,25.55,16719800,VRE
72025,2025-05-29,27.65,27.70,26.80,26.80,16071500,VRE


In [49]:
INDICATORS = ['macd',
               'rsi_30',
               'cci_30',
               'dx_30']
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_turbulence=True,
                     user_defined_feature = False)

processed_done = fe.preprocess_data(processed)
processed_done = processed.copy()
processed_done = processed.fillna(0)
processed_done = processed.replace(np.inf,0)
processed_done


Successfully added technical indicators
Successfully added turbulence index


,date,close,high,low,open,volume,tic
0,2014-01-02,2.29,2.29,2.28,2.29,146982,ACB
1,2014-01-03,2.31,2.31,2.28,2.29,98656,ACB
2,2014-01-06,2.29,2.31,2.28,2.28,190009,ACB
3,2014-01-07,2.32,2.35,2.29,2.31,152905,ACB
4,2014-01-08,2.31,2.32,2.29,2.32,58852,ACB
...,...,...,...,...,...,...,...
72022,2025-05-26,25.50,25.65,24.50,25.60,8734200,VRE
72023,2025-05-27,25.30,25.75,25.25,25.50,11529600,VRE
72024,2025-05-28,26.70,26.80,25.35,25.55,16719800,VRE
72025,2025-05-29,27.65,27.70,26.80,26.80,16071500,VRE


In [46]:
# assuming `processed` has columns ['date','tic',...,'turbulence']
turb_per_date = (
    processed_done[['date','turbulence']]
    .drop_duplicates('date')
    .set_index('date')
    .sort_index()
)

# now turb_per_date is one row per date:
print(turb_per_date.sample(20))

KeyError: "['turbulence'] not in index"

In [65]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# … your data prep up through df_cumulative …

start, end = "2020-01-02", "2020-03-30"
df_win = df_cumulative.loc[start:end]

# Make turbulence a Series, not DataFrame
turb_win = turb_per_date['turbulence'].loc[start:end]
turb_win.index = pd.to_datetime(turb_win.index)

fig, ax = plt.subplots(figsize=(15, 5))
df_win.plot(ax=ax, linewidth=1.5)
ax.set_xlim(start, end)

ax.xaxis.set_major_locator(mdates.DayLocator(interval=9))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
ax.grid(axis='x', which='major', linestyle='--', linewidth=0.5)
ax.grid(axis='y', which='major', linestyle='--', linewidth=0.5)

ax.set_ylabel('Cumulative Return (%)')
ax.legend(['Ensemble', 'PPO', 'A2C', 'DJIA', 'Min var'])

# Twin axis — now turb_win is a Series
ax2 = ax.twinx()
ax2.plot(turb_win.index, turb_win.values,
         color='red', linestyle='--', linewidth=2,
         label='Turbulence Index')
ax2.set_ylabel('Turbulence Index')

# This will now be a single float
ax2.set_ylim(0, turb_win.max() * 1.1)

# Merge legends
lines, labs = ax.get_legend_handles_labels()
l2, la2 = ax2.get_legend_handles_labels()
ax.legend(lines + l2, labs + la2, loc='lower left', frameon=False)

plt.tight_layout()
plt.show()
plt.savefig('turbulence_newa2cppo.pdf')
